In [0]:
from pyspark.sql.functions import when, col, to_date
from pyspark.sql import functions as F
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
import seaborn as sns
import pandas as pd
import numpy as np
import math
import re


In [0]:
!apt-get update -qq
!apt-get install -y fonts-nanum

In [0]:
font_dirs = ["/usr/share/fonts/truetype/nanum/"]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
 
for font_file in font_files:
    font_manager.fontManager.addfont(font_file)
 
plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)
 
pd.Series([-1,2,3]).plot(title='테스트', figsize=(3,2))
pass

In [0]:
# 처리할 # 타겟 테이블 가져오기
ps_df = spark.read.table("database_03_cache.use_after_month_df")


### 1. 결측치 처리 & 데이터형 변환

In [0]:
from pyspark.sql.functions import when, col, to_date

#for c in ps_df.columns[2:]:
#    ps_df = ps_df.withColumn(c, col(c).cast("float"))

In [0]:
display(ps_df)

Databricks data profile. Run in Databricks to view.

### 2. 컬럼 세분화

#### 2.0 기간 포함/불포함 구분

- 전체 : 12개
- 분류 : 기간 포함된 컬럼 (0개) / 불포함된 컬럼 (12개)

In [0]:
ps_columns = ps_df.columns
len(ps_columns)

In [0]:
### Step 1: 기간이 포함된 컬럼 정규표현식
period_pattern = re.compile(r'^(.*)_(B\d+M|R\d+M)$') # B나 R기간

### Step 2: prefix-period 딕셔너리 만들기
prefix_period_map = {}  # 기간별로 prefix를 저장할 딕셔너리
non_matching_cols = []  # 기간이 포함되지 않은 컬럼

for col in ps_columns:
    match = period_pattern.match(col)
    # prefix(ex.'이용금액_신용')와 기간(ex.'R12M')을 따로 저장
    if match:
        prefix = match.group(1) # ex. '이용금액_신용'
        period = match.group(2) # ex. 'R12M'
        if prefix not in prefix_period_map:
            prefix_period_map[prefix] = {}
        prefix_period_map[prefix][period] = col
    else:
        non_matching_cols.append(col)

print(12 - len(non_matching_cols), prefix_period_map)
print()
print(len(non_matching_cols), non_matching_cols)

#### 2.1 기간 포함 컬럼 (없으므로 생략)
(prefix_period_map 에 담겨져 있음)

> **기간 unique 정리**
| 기간코드   | 의미                                               |
| ------ | ------------------------------------------------ |
| `B0M`  | **현재 기준 시점** (예: 분석 기준이 2025년 5월이면 → 2025년 5월)   |
| `B1M`  | **1개월 전** 기준 (예: 2025년 4월)                       |
| `B2M`  | **2개월 전** 기준 (예: 2025년 3월)                       |
| `R3M`  | **최근 3개월 평균치** (예: 2025년 3\~5월의 평균 이용금액)         |
| `R6M`  | **최근 6개월 평균치** (예: 2024년 12월\~2025년 5월의 평균 이용금액) |
| `R12M` | **최근 12개월 평균치** (예: 2024년 6월\~2025년 5월의 평균 이용금액) |


---

#### 2.2 기간 불포함 컬럼

In [0]:
print(len(non_matching_cols))
notperiod_df = ps_df[non_matching_cols]

#### 💡 데이터 추출

In [0]:
### 데이터 베이스 사용 설정
notperiod_df = notperiod_df.cache()
spark.sql("USE database_03_cache")
print("현재 데이터베이스를 'database_03_cache'로 설정")

### 저장할 테이블 값 입력
#notperiod_df.write.mode("overwrite").saveAsTable("use_after_month_notperiod_df")
print("이용금액(기간 불포함) 관련 테이블 생성 완료")

#### 💡다시 불러오기

In [0]:
### 저장한 테이블 값 입력
ps_notperiod_df = spark.read.table("database_03_cache.use_after_month_df")

In [0]:
ps_notperiod_columns = ps_notperiod_df.columns

In [0]:
display(ps_notperiod_df)

Databricks data profile. Run in Databricks to view.

In [0]:
numeric_cols = [c for c in ps_notperiod_columns if c not in ['기준년월', '발급회원번호']]

#### 이상치 처리/스케일링 - 로그 변환
테이블 분석에서 box plot 확인 결과 대부분 positive skew로 이상치 많은 분포임. 따라서 로그 변환

In [0]:
from pyspark.sql.functions import log1p, col

# 로그 변환
for col_name in numeric_cols:
    ps_notperiod_df = ps_notperiod_df.withColumn(col_name, log1p(col(col_name)))

In [0]:
display(ps_notperiod_df)

Databricks data profile. Run in Databricks to view.

#### 상관관계 분석 (fin)
pyspark.ml.stat.Correlation은 **벡터 열**(아래 코드에서 features변수)에에서만 작동하므로<br>
→ 반드시 VectorAssembler 사용해야 함

**상관관계 유형 설멍**

| 구분    | 피어슨 (Pearson)          | 스피어만 (Spearman)                        |
| ----- | ---------------------- | -------------------------------------- |
| 정의    | 변수 간의 **선형 관계** 측정     | 변수 간의 **순위 기반(모노톤) 관계** 측정             |
| 전제 조건 | 연속형 변수 + 정규분포 근처       | 순위로 바꿔도 의미 있는 데이터                      |
| 민감도   | 이상치에 민감                | 이상치에 강건                                |
| 사용 예  | 소비금액처럼 **정량적인 값 간 관계** | **비선형적이지만 단조적인 관계** (ex. 만족도 등급 vs 소비 등급) |

✅ 우리 분석 목적엔?
- "소비 금액의 절대 크기"를 분석하고 싶다면 → 피어슨 (소비 크기에 따른 상품 추천)
-  "어디에 더 많이 쓰는지 성향"을 보고 싶다면 → 스피어만(소비 성향 기반 클러스터링)

In [0]:
import math
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col

# 통계적으로 유의한 샘플 크기 계산
def calculate_sample_size(population_size, confidence_level=0.95, margin_error=0.05):
    """
    통계적으로 유의한 샘플 크기 계산
    """
    z_score = 2.576  # 99% 신뢰도
    p = 0.5  # 최대 분산
    
    n = (z_score**2 * p * (1-p)) / (margin_error**2)
    n_adjusted = n / (1 + (n-1)/population_size)
    
    return int(n_adjusted)

# 데이터 로드
ps_df = ps_notperiod_df
total_count = ps_df.count()

# 통계적 샘플 크기 계산
sample_size = calculate_sample_size(total_count)
sample_fraction = sample_size / total_count

print(f"전체 데이터: {total_count:,}")
print(f"필요 샘플 크기: {sample_size:,}")
print(f"샘플링 비율: {sample_fraction:.4f}")

In [0]:
# 2. 층화 샘플링 (기준년월별로 균등하게)
def stratified_sampling(df, strata_col="기준년월", sample_fraction=0.01):
    """
    층화 샘플링으로 대표성 있는 샘플 생성
    """
    # 각 층(기준년월)별 샘플링
    strata_samples = []
    
    for month in df.select(strata_col).distinct().collect():
        month_value = month[strata_col]
        month_df = df.filter(col(strata_col) == month_value)
        month_sample = month_df.sample(fraction=sample_fraction, seed=42)
        strata_samples.append(month_sample)
    
    # 모든 층 합치기
    final_sample = strata_samples[0]
    for sample in strata_samples[1:]:
        final_sample = final_sample.union(sample)
    
    return final_sample

# 층화 샘플링 실행
print("=== 층화 샘플링 실행 ===")
sampled_df = stratified_sampling(ps_df, sample_fraction=0.005)  # 0.5%
sampled_count = sampled_df.count()
print(f"샘플 데이터: {sampled_count:,}")

In [0]:
# 3. 빠른 상관관계 분석 (피쳐 수 제한 없음)
def fast_correlation_analysis(df):

    # 수치형 컬럼 선택
    numeric_cols = [col_name for col_name, data_type in df.dtypes
                    if data_type in ['int', 'bigint', 'float', 'double']]
    
    # 키 컬럼 제외
    exclude_cols = ['기준년월', '발급회원번호']
    analysis_cols = [col for col in numeric_cols if col not in exclude_cols]
    
    print(f"분석할 피처 수: {len(analysis_cols)}")
    
    # null 처리
    df_filled = df.fillna(0, subset=analysis_cols)
    
    # 벡터화
    assembler = VectorAssembler(inputCols=analysis_cols, outputCol="features")
    vector_df = assembler.transform(df_filled).select("features")
    
    # 캐싱
    vector_df.cache()
    vector_df.count()
    
    # 상관관계 계산
    print("상관관계 계산 중...")
    correlation_matrix = Correlation.corr(vector_df, "features", method="pearson").head()[0]
    
    return correlation_matrix, analysis_cols

# 빠른 분석 실행 (모든 피처 사용)
correlation_matrix, feature_names = fast_correlation_analysis(sampled_df)
print("상관관계 계산 완료!")

In [0]:
# 4. 결과 분석 및 시각화
import numpy as np
import pandas as pd

# 상관관계 매트릭스를 numpy 배열로 변환
corr_array = correlation_matrix.toArray()

# 높은 상관관계 찾기
high_correlations = []
for i in range(len(feature_names)):
    for j in range(i+1, len(feature_names)):
        corr_value = corr_array[i][j]
        if abs(corr_value) > 0.7:  # 0.7 이상
            high_correlations.append({
                'feature1': feature_names[i],
                'feature2': feature_names[j],
                'correlation': corr_value
            })

# 결과 출력
print(f"\n=== 높은 상관관계 ({len(high_correlations)}개) ===")
high_correlations_sorted = sorted(high_correlations, 
                                 key=lambda x: abs(x['correlation']), 
                                 reverse=True)

for corr in high_correlations_sorted[:10]:
    print(f"{corr['feature1']} ↔ {corr['feature2']}: {corr['correlation']:.3f}")

In [0]:
# 5.히트맵 생성
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # 상관관계 매트릭스를 DataFrame으로 변환
    corr_df = pd.DataFrame(corr_array, 
                          index=feature_names, 
                          columns=feature_names)
    
    print(f"DataFrame 크기: {corr_df.shape}")
    print(f"DataFrame 인덱스 수: {len(corr_df.index)}")
    print(f"DataFrame 컬럼 수: {len(corr_df.columns)}")
    
    # 큰 히트맵을 위한 설정
    plt.figure(figsize=(20, 18))  # 크기 증가
    
    # 히트맵 생성 (라벨 크기 조정)
    sns.heatmap(corr_df, 
                annot=False,  # 숫자 표시 끄기 (너무 많아서)
                cmap='coolwarm', 
                center=0,
                square=True, 
                fmt='.2f',
                xticklabels=True,  # x축 라벨 표시
                yticklabels=True,  # y축 라벨 표시
                cbar_kws={'shrink': 0.8})
    
    # 라벨 크기 조정
    plt.xticks(rotation=45, ha='right', fontsize=8)
    plt.yticks(rotation=0, fontsize=8)
    plt.title('Feature Correlation Heatmap (All Features)', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    # 상관관계가 높은 피처들만 별도 히트맵
    print("\n=== 높은 상관관계 피처들만 히트맵 ===")
    
    # 높은 상관관계를 가진 피처들 찾기
    high_corr_features = set()
    threshold = 0.7
    
    for i in range(len(feature_names)):
        for j in range(i+1, len(feature_names)):
            if abs(corr_array[i][j]) > threshold:
                high_corr_features.add(feature_names[i])
                high_corr_features.add(feature_names[j])
    
    if high_corr_features:
        high_corr_features = list(high_corr_features)
        print(f"높은 상관관계 피처 수: {len(high_corr_features)}")
        
        # 서브셋 히트맵
        corr_subset = corr_df.loc[high_corr_features, high_corr_features]
        
        plt.figure(figsize=(12, 10))
        sns.heatmap(corr_subset, 
                    annot=False, 
                    cmap='coolwarm', 
                    center=0,
                    square=True, 
                    fmt='.2f',
                    xticklabels=True,
                    yticklabels=True)
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.title(f'High Correlation Features Heatmap (>{threshold})')
        plt.tight_layout()
        plt.show()
    else:
        print("높은 상관관계를 가진 피처가 없습니다.")
        
except ImportError:
    print("matplotlib/seaborn이 없어 히트맵을 생성할 수 없습니다.")
except Exception as e:
    print(f"히트맵 생성 중 오류: {str(e)}")

In [0]:
# 6. 다중공선성 검사
def check_multicollinearity(corr_matrix, feature_names, threshold=0.9):
    """
    다중공선성 검사
    """
    corr_array = corr_matrix.toArray()
    multicollinear_pairs = []
    
    for i in range(len(feature_names)):
        for j in range(i+1, len(feature_names)):
            corr_value = abs(corr_array[i][j])
            if corr_value > threshold:
                multicollinear_pairs.append({
                    'feature1': feature_names[i],
                    'feature2': feature_names[j],
                    'correlation': corr_array[i][j]
                })
    
    return multicollinear_pairs

# 다중공선성 검사
multicollinear = check_multicollinearity(correlation_matrix, feature_names, 0.9)

print(f"\n=== 다중공선성 위험 ({len(multicollinear)}개) ===")
for pair in multicollinear:
    print(f"⚠️ {pair['feature1']} ↔ {pair['feature2']}: {pair['correlation']:.3f}")

#### 상관관계 분석 (GPU)

In [0]:
import torch
import numpy as np
import pandas as pd
from pyspark.sql.functions import col
import time

print("=== GPU로 상관관계 분석 ===")

# 1. GPU 메모리 최적화 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

if torch.cuda.is_available():
    gpu_props = torch.cuda.get_device_properties(0)
    total_memory = gpu_props.total_memory / 1024**3
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 총 메모리: {total_memory:.1f} GB")
    torch.cuda.empty_cache()

# 2. 전체 데이터 로드
ps_df = ps_notperiod_df
numeric_cols = [col_name for col_name, data_type in ps_df.dtypes
                if data_type in ['int', 'bigint', 'float', 'double']]

total_rows = ps_df.count()
n_features = len(numeric_cols)
print(f"전체 데이터 수: {total_rows:,}")
print(f"분석할 피처 수: {n_features}")

def compute_large_chunk_correlation(df, numeric_cols, device):
    """
    대용량 청크로 GPU 상관관계 계산 (메모리 활용도 극대화)
    """
    print("\n=== 대용량 청크 GPU 처리 시작 ===")
    
    # 훨씬 큰 청크 크기 설정 (Tesla T4 16GB 기준)
    if torch.cuda.is_available():
        # 16GB GPU에서 안전하게 사용할 수 있는 크기
        # 상관관계 매트릭스 계산 시 중간 결과물 고려하여 보수적으로 설정
        large_chunk_size = 2000000  # 200만 행부터 시작
        
        # 메모리 사용량 추정
        estimated_memory_gb = (large_chunk_size * n_features * 4) / 1024**3  # float32 기준
        print(f"청크당 예상 메모리 사용량: {estimated_memory_gb:.2f} GB")
        
        # GPU 메모리의 70% 이상 사용하도록 조정
        target_memory_usage = total_memory * 0.7  # 70% 사용 목표
        optimal_chunk_size = int((target_memory_usage * 1024**3) / (n_features * 4 * 3))  # 안전 마진
        
        # 최종 청크 크기 결정 (최소 100만, 최대 500만)
        final_chunk_size = max(1000000, min(optimal_chunk_size, 5000000))
        
    else:
        final_chunk_size = 1000000  # CPU의 경우
    
    print(f"최종 청크 크기: {final_chunk_size:,} 행")
    
    # 청크 개수 계산
    n_rows = df.count()
    n_chunks = max(1, (n_rows + final_chunk_size - 1) // final_chunk_size)
    
    print(f"총 {n_chunks}개 청크로 분할")
    
    if n_chunks > 10:
        print("⚠️ 청크 개수가 많습니다. 청크 크기를 더 늘려보겠습니다.")
        final_chunk_size = max(final_chunk_size, n_rows // 5)  # 최대 5개 청크로 제한
        n_chunks = max(1, (n_rows + final_chunk_size - 1) // final_chunk_size)
        print(f"조정된 청크 크기: {final_chunk_size:,} 행")
        print(f"조정된 청크 개수: {n_chunks}개")
    
    # 상관관계 매트릭스 누적을 위한 변수들
    correlation_sum = None
    total_weight = 0
    
    for chunk_idx in range(n_chunks):
        chunk_start_time = time.time()
        
        print(f"\n{'='*50}")
        print(f"청크 {chunk_idx + 1}/{n_chunks} 처리 중...")
        
        # 청크 데이터 추출 (더 효율적인 방법)
        if n_chunks == 1:
            # 전체 데이터를 한 번에 처리
            chunk_df = df
        else:
            # 분할 처리
            chunk_fraction = 1.0 / n_chunks
            chunk_df = df.sample(fraction=chunk_fraction, seed=42 + chunk_idx)
        
        # Pandas로 변환
        print("  PySpark → Pandas 변환 중...")
        conversion_start = time.time()
        chunk_pdf = chunk_df.select(*numeric_cols).fillna(0).toPandas()
        conversion_time = time.time() - conversion_start
        
        actual_chunk_size = len(chunk_pdf)
        print(f"  실제 청크 크기: {actual_chunk_size:,} x {len(chunk_pdf.columns)}")
        print(f"  변환 시간: {conversion_time:.2f}초")
        
        if actual_chunk_size == 0:
            continue
        
        try:
            # GPU 메모리 상태 확인
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                initial_memory = torch.cuda.memory_allocated() / 1024**3
                available_memory = (torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated()) / 1024**3
                print(f"  사용 가능 GPU 메모리: {available_memory:.2f} GB")
            
            # GPU 텐서로 변환
            print("  GPU 텐서 변환 중...")
            tensor_start = time.time()
            chunk_tensor = torch.tensor(chunk_pdf.values, dtype=torch.float32).to(device)
            tensor_time = time.time() - tensor_start
            
            if torch.cuda.is_available():
                after_tensor_memory = torch.cuda.memory_allocated() / 1024**3
                memory_used = after_tensor_memory - initial_memory
                print(f"  GPU 메모리 사용량: {memory_used:.2f} GB")
                print(f"  텐서 변환 시간: {tensor_time:.2f}초")
            
            # 상관관계 계산
            print("  상관관계 계산 중...")
            corr_start = time.time()
            chunk_correlation = torch.corrcoef(chunk_tensor.T)
            corr_time = time.time() - corr_start
            print(f"  상관관계 계산 시간: {corr_time:.2f}초")
            
            # CPU로 이동하여 누적
            chunk_corr_cpu = chunk_correlation.cpu().numpy()
            
            # 가중 평균으로 누적
            weight = actual_chunk_size
            if correlation_sum is None:
                correlation_sum = chunk_corr_cpu * weight
            else:
                correlation_sum += chunk_corr_cpu * weight
            total_weight += weight
            
            # 메모리 정리
            del chunk_tensor, chunk_correlation, chunk_pdf
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
            chunk_total_time = time.time() - chunk_start_time
            print(f"  청크 총 처리 시간: {chunk_total_time:.2f}초")
            print(f"  진행률: {(chunk_idx + 1) / n_chunks * 100:.1f}%")
            
            # 남은 시간 추정
            if chunk_idx > 0:
                avg_time_per_chunk = (time.time() - start_time) / (chunk_idx + 1)
                remaining_time = avg_time_per_chunk * (n_chunks - chunk_idx - 1)
                print(f"  예상 남은 시간: {remaining_time:.1f}초 ({remaining_time/60:.1f}분)")
            
        except RuntimeError as e:
            if "out of memory" in str(e):
                print(f"  ❌ GPU 메모리 부족! 현재 청크 크기: {actual_chunk_size:,}")
                print("  더 작은 청크로 재시도하거나 CPU로 fallback이 필요합니다.")
                raise e
            else:
                raise e
    
    # 최종 상관관계 매트릭스 계산
    if total_weight > 0:
        final_correlation = correlation_sum / total_weight
        return final_correlation, numeric_cols
    else:
        return None, numeric_cols

# 3. 대용량 청크 상관관계 분석 실행
print(f"\n🚀 대용량 청크로 전체 데이터 {total_rows:,}행 분석 시작")
start_time = time.time()

try:
    correlation_matrix, feature_names = compute_large_chunk_correlation(
        ps_df, 
        numeric_cols, 
        device
    )
    
    end_time = time.time()
    total_time = end_time - start_time
    print(f"\n⏱️ 전체 처리 시간: {total_time:.2f}초 ({total_time/60:.1f}분)")
    print(f"📊 처리 속도: {total_rows/total_time:,.0f} 행/초")
    
    # 4. 결과 분석
    if correlation_matrix is not None:
        print(f"\n=== ✅ 대용량 청크 상관관계 분석 결과 ✅ ===")
        print(f"분석된 데이터: {total_rows:,}행")
        print(f"분석된 피처: {len(feature_names)}개")
        print(f"상관관계 매트릭스 크기: {correlation_matrix.shape}")
        
        # 높은 상관관계 분석
        def analyze_correlations(corr_matrix, features, threshold=0.7):
            high_corr = []
            n = len(features)
            for i in range(n):
                for j in range(i+1, n):
                    corr_val = corr_matrix[i, j]
                    if abs(corr_val) > threshold:
                        high_corr.append({
                            'feature1': features[i],
                            'feature2': features[j],
                            'correlation': float(corr_val)
                        })
            return high_corr
        
        # 높은 상관관계 출력
        high_correlations = analyze_correlations(correlation_matrix, feature_names, 0.7)
        print(f"\n높은 상관관계 (|r| > 0.7): {len(high_correlations)}개")
        
        for pair in sorted(high_correlations, key=lambda x: abs(x['correlation']), reverse=True)[:20]:
            print(f"  {pair['feature1']} ↔ {pair['feature2']}: {pair['correlation']:.3f}")
        
        # 다중공선성 위험
        multicollinear = analyze_correlations(correlation_matrix, feature_names, 0.9)
        print(f"\n다중공선성 위험 (|r| > 0.9): {len(multicollinear)}개")
        
        for pair in sorted(multicollinear, key=lambda x: abs(x['correlation']), reverse=True):
            print(f"  ⚠️ {pair['feature1']} ↔ {pair['feature2']}: {pair['correlation']:.3f}")

except Exception as e:
    print(f"❌ 오류 발생: {str(e)}")
    print("CPU로 fallback을 시도하거나 청크 크기를 더 줄여보세요.")

# 5. 메모리 정리
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    final_memory = torch.cuda.memory_allocated() / 1024**3
    peak_memory = torch.cuda.max_memory_allocated() / 1024**3
    print(f"\n🧹 최종 GPU 메모리 사용량: {final_memory:.2f} GB")
    print(f"📈 최대 GPU 메모리 사용량: {peak_memory:.2f} GB")

print("\n✅ GPU 상관관계 분석 완료!")

#### 다중공신성 제거

In [0]:
def solve_use_after_month_multicollinearity(ps_notperiod_df):
    """
    USE_AFTER_MONTH 테이블 다중공선성 해결
    """
    print("=== USE_AFTER_MONTH 다중공선성 해결 ===")
    print("="*50)
    
    # 다중공선성 분석 및 삭제 전략
    multicollinearity_drops = {
        # 1. 신용카드 계층 구조 (신용 > 신판)
        "신용카드_계층": [
            "이용후경과월_신판"  # 이용후경과월_신용과 0.980 상관관계
        ],
        
        # 2. 할부 관련 (전체 할부 > 무이자 할부)
        "할부_중복": [
            "이용후경과월_할부_무이자"  # 이용후경과월_할부와 0.954 상관관계
        ]
    }
    
    # 전체 삭제 컬럼 리스트
    all_drops = []
    for category, cols in multicollinearity_drops.items():
        all_drops.extend(cols)
    
    print("📋 다중공선성 해결 전략:")
    print("-" * 40)
    
    total_drops = 0
    for category, cols in multicollinearity_drops.items():
        print(f"\n🔸 {category} ({len(cols)}개):")
        for i, col in enumerate(cols, 1):
            print(f"   {i}. {col}")
        total_drops += len(cols)
    
    print(f"\n📊 삭제 요약:")
    print(f"   총 삭제 대상: {total_drops}개 컬럼")
    
    # 실제 존재하는 컬럼만 필터링
    existing_drops = [col for col in all_drops if col in ps_notperiod_df.columns]
    missing_cols = [col for col in all_drops if col not in ps_notperiod_df.columns]
    
    print(f"   실제 존재: {len(existing_drops)}개")
    print(f"   존재하지 않음: {len(missing_cols)}개")
    
    if missing_cols:
        print(f"\n⚠️ 존재하지 않는 컬럼들:")
        for col in missing_cols:
            print(f"   - {col}")
    
    print(f"\n🔧 실제 삭제할 컬럼들:")
    for i, col in enumerate(existing_drops, 1):
        print(f"   {i}. {col}")
    
    # 삭제 실행
    if existing_drops:
        ps_df_cleaned = ps_notperiod_df.drop(*existing_drops)
        
        print(f"\n✅ 다중공선성 해결 완료!")
        print(f"   삭제 전: {len(ps_notperiod_df.columns)} 컬럼")
        print(f"   삭제 후: {len(ps_df_cleaned.columns)} 컬럼")
        print(f"   실제 삭제: {len(existing_drops)} 컬럼")
        print(f"   삭제 비율: {len(existing_drops)/len(ps_notperiod_df.columns)*100:.1f}%")
        
        # 해결된 다중공선성 쌍들
        print(f"\n📋 해결된 다중공선성 쌍들 (2개):")
        resolved_pairs = [
            "이용후경과월_신용 ↔ 이용후경과월_신판 (0.980)",
            "이용후경과월_할부 ↔ 이용후경과월_할부_무이자 (0.954)"
        ]
        
        for i, pair in enumerate(resolved_pairs, 1):
            print(f"   {i}. ✅ {pair}")
        
        print(f"\n🎯 해결 논리:")
        print(f"   • 신판카드는 신용카드의 하위 개념이므로 신판 삭제")
        print(f"   • 할부_무이자는 전체 할부에 포함되므로 무이자 삭제")
        
        return ps_df_cleaned, existing_drops
    else:
        print("\n❌ 삭제할 컬럼이 없습니다.")
        return ps_notperiod_df, []

def validate_use_after_month_solution(ps_df_cleaned, deleted_cols):
    """
    USE_AFTER_MONTH 다중공선성 해결 효과 검증
    """
    print(f"\n{'='*50}")
    print("🔍 USE_AFTER_MONTH 다중공선성 해결 검증")
    print(f"{'='*50}")
    
    print(f"📋 삭제된 컬럼들:")
    for i, col in enumerate(deleted_cols, 1):
        print(f"   {i}. {col}")
    
    # 간단한 샘플 검증
    sample_df = ps_df_cleaned.sample(fraction=0.001, seed=42)
    numeric_cols = [col_name for col_name, data_type in sample_df.dtypes
                    if data_type in ['int', 'bigint', 'float', 'double']]
    
    if len(numeric_cols) > 1:
        print(f"\n📊 검증 결과:")
        print(f"   남은 수치형 컬럼: {len(numeric_cols)}개")
        
        try:
            # Pandas 변환 후 간단한 상관관계 체크
            sample_pdf = sample_df.select(*numeric_cols).fillna(0).toPandas()
            corr_matrix = sample_pdf.corr()
            
            # 다중공선성 위험 카운트
            very_high_count = 0
            high_count = 0
            
            n = len(corr_matrix.columns)
            for i in range(n):
                for j in range(i+1, n):
                    corr_val = abs(corr_matrix.iloc[i, j])
                    if corr_val > 0.9:
                        very_high_count += 1
                    elif corr_val > 0.7:
                        high_count += 1
            
            print(f"   다중공선성 위험 (>0.9): {very_high_count}개")
            print(f"   높은 상관관계 (0.7-0.9): {high_count}개")
            
            if very_high_count == 0:
                print("   ✅ 다중공선성 위험이 완전히 해결되었습니다!")
            else:
                print("   ⚠️ 일부 다중공선성이 남아있을 수 있습니다.")
                
        except Exception as e:
            print(f"   ⚠️ 검증 중 오류: {str(e)}")
    
    return ps_df_cleaned

# 실행
print("🚀 USE_AFTER_MONTH 다중공선성 해결 시작")
print("="*60)

# 다중공선성 해결
ps_df_cleaned, deleted_cols = solve_use_after_month_multicollinearity(ps_notperiod_df)

# 효과 검증
ps_df_final = validate_use_after_month_solution(ps_df_cleaned, deleted_cols)

print(f"\n🎯 USE_AFTER_MONTH 다중공선성 해결 완료!")
print(f"   삭제된 컬럼: {len(deleted_cols)}개")
print(f"   해결된 다중공선성 쌍: 2개")
print(f"   최종 컬럼 수: {len(ps_df_final.columns)}개")

In [0]:
import torch
import numpy as np
import pandas as pd
from pyspark.sql.functions import col
import time

print("=== GPU로 상관관계 분석 ===")

# 1. GPU 메모리 최적화 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

if torch.cuda.is_available():
    gpu_props = torch.cuda.get_device_properties(0)
    total_memory = gpu_props.total_memory / 1024**3
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 총 메모리: {total_memory:.1f} GB")
    torch.cuda.empty_cache()

# 2. 전체 데이터 로드
ps_df = ps_df_final
numeric_cols = [col_name for col_name, data_type in ps_df.dtypes
                if data_type in ['int', 'bigint', 'float', 'double']]

total_rows = ps_df.count()
n_features = len(numeric_cols)
print(f"전체 데이터 수: {total_rows:,}")
print(f"분석할 피처 수: {n_features}")

def compute_large_chunk_correlation(df, numeric_cols, device):
    """
    대용량 청크로 GPU 상관관계 계산 (메모리 활용도 극대화)
    """
    print("\n=== 대용량 청크 GPU 처리 시작 ===")
    
    # 훨씬 큰 청크 크기 설정 (Tesla T4 16GB 기준)
    if torch.cuda.is_available():
        # 16GB GPU에서 안전하게 사용할 수 있는 크기
        # 상관관계 매트릭스 계산 시 중간 결과물 고려하여 보수적으로 설정
        large_chunk_size = 2000000  # 200만 행부터 시작
        
        # 메모리 사용량 추정
        estimated_memory_gb = (large_chunk_size * n_features * 4) / 1024**3  # float32 기준
        print(f"청크당 예상 메모리 사용량: {estimated_memory_gb:.2f} GB")
        
        # GPU 메모리의 70% 이상 사용하도록 조정
        target_memory_usage = total_memory * 0.7  # 70% 사용 목표
        optimal_chunk_size = int((target_memory_usage * 1024**3) / (n_features * 4 * 3))  # 안전 마진
        
        # 최종 청크 크기 결정 (최소 100만, 최대 500만)
        final_chunk_size = max(1000000, min(optimal_chunk_size, 5000000))
        
    else:
        final_chunk_size = 1000000  # CPU의 경우
    
    print(f"최종 청크 크기: {final_chunk_size:,} 행")
    
    # 청크 개수 계산
    n_rows = df.count()
    n_chunks = max(1, (n_rows + final_chunk_size - 1) // final_chunk_size)
    
    print(f"총 {n_chunks}개 청크로 분할")
    
    if n_chunks > 10:
        print("⚠️ 청크 개수가 많습니다. 청크 크기를 더 늘려보겠습니다.")
        final_chunk_size = max(final_chunk_size, n_rows // 5)  # 최대 5개 청크로 제한
        n_chunks = max(1, (n_rows + final_chunk_size - 1) // final_chunk_size)
        print(f"조정된 청크 크기: {final_chunk_size:,} 행")
        print(f"조정된 청크 개수: {n_chunks}개")
    
    # 상관관계 매트릭스 누적을 위한 변수들
    correlation_sum = None
    total_weight = 0
    
    for chunk_idx in range(n_chunks):
        chunk_start_time = time.time()
        
        print(f"\n{'='*50}")
        print(f"청크 {chunk_idx + 1}/{n_chunks} 처리 중...")
        
        # 청크 데이터 추출 (더 효율적인 방법)
        if n_chunks == 1:
            # 전체 데이터를 한 번에 처리
            chunk_df = df
        else:
            # 분할 처리
            chunk_fraction = 1.0 / n_chunks
            chunk_df = df.sample(fraction=chunk_fraction, seed=42 + chunk_idx)
        
        # Pandas로 변환
        print("  PySpark → Pandas 변환 중...")
        conversion_start = time.time()
        chunk_pdf = chunk_df.select(*numeric_cols).fillna(0).toPandas()
        conversion_time = time.time() - conversion_start
        
        actual_chunk_size = len(chunk_pdf)
        print(f"  실제 청크 크기: {actual_chunk_size:,} x {len(chunk_pdf.columns)}")
        print(f"  변환 시간: {conversion_time:.2f}초")
        
        if actual_chunk_size == 0:
            continue
        
        try:
            # GPU 메모리 상태 확인
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                initial_memory = torch.cuda.memory_allocated() / 1024**3
                available_memory = (torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated()) / 1024**3
                print(f"  사용 가능 GPU 메모리: {available_memory:.2f} GB")
            
            # GPU 텐서로 변환
            print("  GPU 텐서 변환 중...")
            tensor_start = time.time()
            chunk_tensor = torch.tensor(chunk_pdf.values, dtype=torch.float32).to(device)
            tensor_time = time.time() - tensor_start
            
            if torch.cuda.is_available():
                after_tensor_memory = torch.cuda.memory_allocated() / 1024**3
                memory_used = after_tensor_memory - initial_memory
                print(f"  GPU 메모리 사용량: {memory_used:.2f} GB")
                print(f"  텐서 변환 시간: {tensor_time:.2f}초")
            
            # 상관관계 계산
            print("  상관관계 계산 중...")
            corr_start = time.time()
            chunk_correlation = torch.corrcoef(chunk_tensor.T)
            corr_time = time.time() - corr_start
            print(f"  상관관계 계산 시간: {corr_time:.2f}초")
            
            # CPU로 이동하여 누적
            chunk_corr_cpu = chunk_correlation.cpu().numpy()
            
            # 가중 평균으로 누적
            weight = actual_chunk_size
            if correlation_sum is None:
                correlation_sum = chunk_corr_cpu * weight
            else:
                correlation_sum += chunk_corr_cpu * weight
            total_weight += weight
            
            # 메모리 정리
            del chunk_tensor, chunk_correlation, chunk_pdf
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
            chunk_total_time = time.time() - chunk_start_time
            print(f"  청크 총 처리 시간: {chunk_total_time:.2f}초")
            print(f"  진행률: {(chunk_idx + 1) / n_chunks * 100:.1f}%")
            
            # 남은 시간 추정
            if chunk_idx > 0:
                avg_time_per_chunk = (time.time() - start_time) / (chunk_idx + 1)
                remaining_time = avg_time_per_chunk * (n_chunks - chunk_idx - 1)
                print(f"  예상 남은 시간: {remaining_time:.1f}초 ({remaining_time/60:.1f}분)")
            
        except RuntimeError as e:
            if "out of memory" in str(e):
                print(f"  ❌ GPU 메모리 부족! 현재 청크 크기: {actual_chunk_size:,}")
                print("  더 작은 청크로 재시도하거나 CPU로 fallback이 필요합니다.")
                raise e
            else:
                raise e
    
    # 최종 상관관계 매트릭스 계산
    if total_weight > 0:
        final_correlation = correlation_sum / total_weight
        return final_correlation, numeric_cols
    else:
        return None, numeric_cols

# 3. 대용량 청크 상관관계 분석 실행
print(f"\n🚀 대용량 청크로 전체 데이터 {total_rows:,}행 분석 시작")
start_time = time.time()

try:
    correlation_matrix, feature_names = compute_large_chunk_correlation(
        ps_df, 
        numeric_cols, 
        device
    )
    
    end_time = time.time()
    total_time = end_time - start_time
    print(f"\n⏱️ 전체 처리 시간: {total_time:.2f}초 ({total_time/60:.1f}분)")
    print(f"📊 처리 속도: {total_rows/total_time:,.0f} 행/초")
    
    # 4. 결과 분석
    if correlation_matrix is not None:
        print(f"\n=== ✅ 대용량 청크 상관관계 분석 결과 ✅ ===")
        print(f"분석된 데이터: {total_rows:,}행")
        print(f"분석된 피처: {len(feature_names)}개")
        print(f"상관관계 매트릭스 크기: {correlation_matrix.shape}")
        
        # 높은 상관관계 분석
        def analyze_correlations(corr_matrix, features, threshold=0.7):
            high_corr = []
            n = len(features)
            for i in range(n):
                for j in range(i+1, n):
                    corr_val = corr_matrix[i, j]
                    if abs(corr_val) > threshold:
                        high_corr.append({
                            'feature1': features[i],
                            'feature2': features[j],
                            'correlation': float(corr_val)
                        })
            return high_corr
        
        # 높은 상관관계 출력
        high_correlations = analyze_correlations(correlation_matrix, feature_names, 0.7)
        print(f"\n높은 상관관계 (|r| > 0.7): {len(high_correlations)}개")
        
        for pair in sorted(high_correlations, key=lambda x: abs(x['correlation']), reverse=True)[:20]:
            print(f"  {pair['feature1']} ↔ {pair['feature2']}: {pair['correlation']:.3f}")
        
        # 다중공선성 위험
        multicollinear = analyze_correlations(correlation_matrix, feature_names, 0.9)
        print(f"\n다중공선성 위험 (|r| > 0.9): {len(multicollinear)}개")
        
        for pair in sorted(multicollinear, key=lambda x: abs(x['correlation']), reverse=True):
            print(f"  ⚠️ {pair['feature1']} ↔ {pair['feature2']}: {pair['correlation']:.3f}")

except Exception as e:
    print(f"❌ 오류 발생: {str(e)}")
    print("CPU로 fallback을 시도하거나 청크 크기를 더 줄여보세요.")

# 5. 메모리 정리
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    final_memory = torch.cuda.memory_allocated() / 1024**3
    peak_memory = torch.cuda.max_memory_allocated() / 1024**3
    print(f"\n🧹 최종 GPU 메모리 사용량: {final_memory:.2f} GB")
    print(f"📈 최대 GPU 메모리 사용량: {peak_memory:.2f} GB")

print("\n✅ GPU 상관관계 분석 완료!")

In [0]:
### 데이터 베이스 사용 설정
spark.sql("USE database_03_cache")
print("현재 데이터베이스를 'database_03_cache'로 설정")

ps_df_final.write.mode("overwrite").saveAsTable("use_after_month_df_proc")